In [ ]:
# analysis.py
from cmath import exp
import pandas as pd
import numpy as np
from scipy.optimize import curve_fit
import matplotlib as mpl
import matplotlib.pyplot as plt
import os
import json
from pathlib import Path

#  Chapman-Richards function looks like this:
# b0(1-EXP(-b2Age))^b2 
def chapman_richards(age:float, b0:float, b1:float, b2:float):
    return (b0 * (1 - np.exp(-b1*age))) ** b2

# Loop through csv's generated from carbon curves and fit the chapman
# richards function to each set of data.
# The name and paramters are then added to a json like dictionary which
# can be saved in the next cell.
directory = r'..\carbon-images\tables'
files = Path(directory).glob('*.csv')
outjson = {}
for file in files:
    df = pd.read_csv(file,header=None)
    title = str(file).split('\\')[-1].split('.')[-2].replace('-',' ').title()

    x_dummy = df[0]
    y_dummy = df[1]
    pars, cov = curve_fit(f=chapman_richards, maxfev=10000, xdata=x_dummy, ydata=y_dummy, p0=[83, 0.5, 4], bounds=(-np.inf, np.inf))

    
    print('pars',pars,' cov', cov)

    # Edit the font, font size, and axes width
    plt.rcParams['font.size'] = 18
    plt.rcParams['axes.linewidth'] = 2

    # Create figure object and store it in a variable called 'fig'
    fig = plt.figure(figsize=(5, 5))

    # Add axes object to our figure that takes up entire figure
    ax = fig.add_axes([0, 0, 1, 1])


    # Plot the noisy exponential data
    ax.scatter(x_dummy, y_dummy, s=20, color='#00b3b3', label='Data')

    # Add the x and y-axis labels
    ax.set_xlabel('stand age', labelpad=10)
    ax.set_ylabel('AGB', labelpad=10)

    # Set the axis limits
    ax.set_xlim(min(x_dummy), max(x_dummy)+10)
    ax.set_ylim(min(y_dummy), max(y_dummy)+10)

    #plt.savefig('dummy_dataset_exponential.png', dpi=100, bbox_inches='tight')
    plt.title(title)
    plt.show()

    # Edit the font, font size, and axes width
    # mpl.rcParams['font.family'] = 'Avenir'
    plt.rcParams['font.size'] = 18
    plt.rcParams['axes.linewidth'] = 2

    # Create figure object and store it in a variable called 'fig'
    fig = plt.figure(figsize=(5, 5))

    # Add axes object to our figure that takes up entire figure
    ax = fig.add_axes([0, 0, 1, 1])


    # Plot the noisy exponential data
    ax.scatter(x_dummy, y_dummy, s=20, color='#00b3b3', label='Data')
    ax.plot(x_dummy, chapman_richards(x_dummy, *pars), linestyle='--', linewidth=2, color='black')

    # Add the x and y-axis labels
    ax.set_xlabel('Stand Age', labelpad=10)
    ax.set_ylabel('AGB', labelpad=10)

    # Set the axis limits
    ax.set_xlim(min(x_dummy), max(x_dummy)+10)
    ax.set_ylim(min(y_dummy), max(y_dummy)+10)
    plt.title(title)
    plt.show()
    
    #format json
    #"natural_asia_humid": {
    #   "name": "Natural Asia Humid",
    #   "parameters": (22.20201292, 0.08282659, 1.65756327),
    #}
    outjson[title.lower().replace(' ','_')] = {"name":title,"parameters":tuple(pars) }



Optionally save the json. 

In [ ]:
with open('carbon.json', 'w', encoding='utf-8') as f:
    json.dump(outjson, f, ensure_ascii=False, indent=4)


Test the function provides an expected output for a growth curve.

In [ ]:
params = [6.952180603672807, 0.07708632118229979, 2.155374937783795]
biomass = chapman_richards(100, params[0], params[1], params[2])
	
# This will report metric tonnes of carbon per ha (NOT CO2e)
biomass